In [6]:
import { ChatOpenAI } from "@langchain/openai";
import { RunnableSequence } from "@langchain/core/runnables";
import { RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { BufferMemory } from "langchain/memory";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ConversationChain } from "langchain/chains";

In [ ]:
const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
  verbose: true,
});
const memory =new BufferMemory()

const TEMPLATE = `
你是一个乐于助人的 ai 助手。尽你所能回答所有问题。

这是跟人类沟通的聊天历史:
{history}

据此回答人类的问题:
{input}
`;

const prompt= ChatPromptTemplate.fromTemplate(TEMPLATE)

In [ ]:
let tempInput = "";

const chain = RunnableSequence.from([
  {
    input: new RunnablePassthrough(),
    memoryObject: async (input) => {
      const history = await memory.loadMemoryVariables({
        input,
      });
      tempInput = input;
      return history;
    },
  },
  RunnablePassthrough.assign({
    history: (input) => input.memoryObject.history,
  }),
  prompt,
  new RunnablePassthrough({
    func: (input) => console.log(input),
  }),
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (output) => {
      await memory.saveContext(
        {
          input: tempInput,
        },
        {
          output,
        }
      );
    },
  }),
]);


In [ ]:
await chain.invoke("你好, 我叫小明");


In [ ]:
await chain.invoke("我叫什么？");


##实现自定义的 chat history

In [1]:
import { JSONChatHistory } from "./JSONChatHistory/index.ts";
import { AIMessage, HumanMessage } from "@langchain/core/messages";

const history = new JSONChatHistory({
  dir: "chat_data",
  sessionId: "test",
});

await history.addMessages([
  new HumanMessage("Hi, 我叫小明"),
  new AIMessage("你好"),
]);

const messages = await history.getMessages();
console.log(messages);


[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [7]:
const chatModel = new ChatOpenAI({
  configuration: {
    baseURL: "https://api.moonshot.cn/v1",
    apiKey: process.env.OPENAI_API_KEY,
  },
  modelName: "kimi-k2-0711-preview",
  verbose: true,
});
const memory = new BufferMemory({ chatHistory: history });
const chain = new ConversationChain({
  llm: chatModel,
  memory,
});

const res1 = await chain.call({
  input: "我叫什么",
});


[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\nHuman: Hi, 我叫小明\nAI: 你好\nHuman: 我叫什么\nAI:",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/end] [1:llm:ChatOpenAI] [1.50s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "你刚刚说过，你叫小明呀！",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain_core",
            "messages",
            "AIMessage"
          ],
 

In [8]:
const messages = await history.getMessages();
console.log(messages);

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "我叫什么", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "我叫什么",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "你刚刚说过，你叫小明呀！",
      additional_kwargs: {},
      response_metadata